In [2]:
import numpy as np
import pandas as pd
import torch
from pyabc import Distribution, RV
from sbi.utils import BoxUniform

/etc/python/sitecustomize.py:236: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  mod = _original_import(name, globals, locals, fromlist, level)


In [3]:
from src.utils import set_seed
from src.inference import SBIEngine
from models.epidemic_models import simulate_seir  

In [4]:
# 1. Global Setup
set_seed(0) # Fix seed for reproducibility
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

[#] Seed has been fixed to: 0
Using device: cpu


In [5]:
# 2. Define Priors
low_linear = np.array([0.01, 0.01, 0.01])
high_linear = np.array([1.5, 0.5, 0.5])

In [6]:
# 2. Define Priors (Two versions for pyabc and sbi compatibility)
# sbi version (low, high)
sbi_prior = BoxUniform(
    low=torch.tensor([0.01, 0.01, 0.01]),
    high=torch.tensor([1.5, 0.5, 0.5])
)

In [7]:
def generate_training_data(num_sims=100000):
    """Generates training dataset for standard NPE and NPE-LSTM"""
    print(f"[*] Generating {num_sims} simulations for NPE training...")
    thetas = sbi_prior.sample((num_sims,))
    xs = []
    for t in thetas:
        data = simulate_seir(t.numpy(), [99990, 0, 10, 0], duration=100, noise_type="none")
        xs.append(torch.tensor(data, dtype=torch.float32))
    return thetas, torch.stack(xs)

In [8]:
# thetas_train, x_train = generate_training_data()

In [9]:
import pickle

In [10]:
# with open("./data/M1_dataset.pkl", "wb") as handle:
#     pickle.dump(x_train, handle)

# with open("./data/M1_params.pkl", "wb") as handle:
#     pickle.dump(thetas_train, handle)

In [11]:
with open("./data/M1_dataset.pkl", "rb") as handle:
    xs_train = pickle.load(handle)

with open("./data/M1_params.pkl", "rb") as handle:
    thetas_train = pickle.load(handle)

In [12]:
with open('./data/model1.pkl', 'rb') as f:
    true_dataset = pickle.load(f)

In [13]:
import src.utils
import time

In [14]:
xs_train=src.utils.add_poisson_noise(xs_train)

In [15]:
engine = SBIEngine(density_estimator='maf')

In [20]:
start_time = time.time()
npe_post_1k, npe_samples_1k = engine.run_npe(
    obs_data = true_dataset[0]['poisson'],
    prior = sbi_prior,
    thetas = torch.tensor(thetas_train[:1000],dtype=torch.float32),
    xs = torch.tensor(xs_train[:1000],dtype=torch.float32),
    use_lstm=False,
    batch_size=64
)
end_time = time.time()
print(f"Done in {end_time - start_time:.2f} seconds")

/tmp/ipykernel_1222862/3462967249.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  thetas = torch.tensor(thetas_train[:1000],dtype=torch.float32),


[*] Running NPE (use_lstm=False) with batch size 64...
 Neural network successfully converged after 51 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 51
        Best validation performance: -4.0120
        -------------------------
        


10360it [00:00, 138196.51it/s]           

Done in 6.98 seconds


In [21]:
with open("./results/Model1/NPE/M1_1k_maf_posterior.pkl", "wb") as handle:
    pickle.dump(npe_post_1k, handle)

In [23]:
start_time = time.time()
npe_post_10k, npe_samples_10k = engine.run_npe(
    obs_data = true_dataset[0]['poisson'],
    prior = sbi_prior,
    thetas = torch.tensor(thetas_train[:10000],dtype=torch.float32),
    xs = torch.tensor(xs_train[:10000],dtype=torch.float32),
    use_lstm=False,
    batch_size=128
)
end_time = time.time()
print(f"Done in {end_time - start_time:.2f} seconds")

/tmp/ipykernel_1222862/1712808748.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  thetas = torch.tensor(thetas_train[:10000],dtype=torch.float32),


[*] Running NPE (use_lstm=False) with batch size 128...
 Neural network successfully converged after 130 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 130
        Best validation performance: -6.5555
        -------------------------
        


10515it [00:00, 136558.19it/s]           

Done in 98.56 seconds


In [24]:
with open("./results/Model1/NPE/M1_10k_maf_posterior.pkl", "wb") as handle:
    pickle.dump(npe_post_10k, handle)

In [25]:
start_time = time.time()
npe_post_100k, npe_samples_100k = engine.run_npe(
    obs_data = true_dataset[0]['poisson'],
    prior = sbi_prior,
    thetas = torch.tensor(thetas_train[:100000],dtype=torch.float32),
    xs = torch.tensor(xs_train[:100000],dtype=torch.float32),
    use_lstm=False,
    batch_size=256
)
end_time = time.time()
print(f"Done in {end_time - start_time:.2f} seconds")

[*] Running NPE (use_lstm=False) with batch size 256...


/tmp/ipykernel_1222862/1266160277.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  thetas = torch.tensor(thetas_train[:100000],dtype=torch.float32),


 Neural network successfully converged after 163 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 163
        Best validation performance: -8.6285
        -------------------------
        


100%|██████████| 10000/10000 [00:00<00:00, 132279.05it/s]

Done in 703.69 seconds


In [26]:
with open("./results/Model1/NPE/M1_100k_maf_posterior.pkl", "wb") as handle:
    pickle.dump(npe_post_100k, handle)

In [27]:
engine = SBIEngine(density_estimator='nsf')

In [28]:
start_time = time.time()
npe_post_1k_nsf, npe_samples_1k = engine.run_npe(
    obs_data = true_dataset[0]['poisson'],
    prior = sbi_prior,
    thetas = torch.tensor(thetas_train[:1000],dtype=torch.float32),
    xs = torch.tensor(xs_train[:1000],dtype=torch.float32),
    use_lstm=False,
    batch_size=64
)
end_time = time.time()
print(f"Done in {end_time - start_time:.2f} seconds")

[*] Running NPE (use_lstm=False) with batch size 64...


/tmp/ipykernel_1222862/1497939270.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  thetas = torch.tensor(thetas_train[:1000],dtype=torch.float32),


 Neural network successfully converged after 83 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 83
        Best validation performance: -4.4945
        -------------------------
        


  0%|          | 0/10000 [00:00<?, ?it/s]/home/gjang12/.conda/envs/sbi_env/lib/python3.11/site-packages/nflows/transforms/lu.py:80: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2190.)
  outputs, _ = torch.triangular_solve(
10137it [00:00, 36455.07it/s]                          

Done in 33.85 seconds


In [29]:
with open("./results/Model1/NPE/M1_1k_nsf_posterior.pkl", "wb") as handle:
    pickle.dump(npe_post_1k_nsf, handle)

In [30]:
start_time = time.time()
npe_post_10k_nsf, npe_samples_10k = engine.run_npe(
    obs_data = true_dataset[0]['poisson'],
    prior = sbi_prior,
    thetas = torch.tensor(thetas_train[:10000],dtype=torch.float32),
    xs = torch.tensor(xs_train[:10000],dtype=torch.float32),
    use_lstm=False,
    batch_size=128
)
end_time = time.time()
print(f"Done in {end_time - start_time:.2f} seconds")

/tmp/ipykernel_1222862/2080014715.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  thetas = torch.tensor(thetas_train[:10000],dtype=torch.float32),


[*] Running NPE (use_lstm=False) with batch size 128...
 Neural network successfully converged after 140 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 140
        Best validation performance: -7.7263
        -------------------------
        


10097it [00:00, 145629.35it/s]           

Done in 288.04 seconds


In [31]:
with open("./results/Model1/NPE/M1_10k_nsf_posterior.pkl", "wb") as handle:
    pickle.dump(npe_post_10k_nsf, handle)

In [32]:
start_time = time.time()
npe_post_100k_nsf, npe_samples_100k = engine.run_npe(
    obs_data = true_dataset[0]['poisson'],
    prior = sbi_prior,
    thetas = torch.tensor(thetas_train[:100000],dtype=torch.float32),
    xs = torch.tensor(xs_train[:100000],dtype=torch.float32),
    use_lstm=False,
    batch_size=256
)
end_time = time.time()
print(f"Done in {end_time - start_time:.2f} seconds")

[*] Running NPE (use_lstm=False) with batch size 256...


/tmp/ipykernel_1222862/4129846634.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  thetas = torch.tensor(thetas_train[:100000],dtype=torch.float32),


 Neural network successfully converged after 144 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 144
        Best validation performance: -9.0451
        -------------------------
        


100%|██████████| 10000/10000 [00:00<00:00, 199233.52it/s]

Done in 1653.59 seconds


In [33]:
with open("./results/Model1/NPE/M1_100k_nsf_posterior.pkl", "wb") as handle:
    pickle.dump(npe_post_100k_nsf, handle)